# PyTorch

## __Tensors__

Одно из основных понятий в PyTorch -- это __Tenosor__. 

https://pytorch.org/docs/master/tensors.html

__Tensor__ -- это такой же массив, как и в __numpy.array__, размерность и тип данных которого мы можем задать. Tensor в отличие от numpy.array может вычисляться на __GPU__.

In [1]:
import numpy as np
import torch

In [2]:
N = 100
D_in = 50

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

x = np.random.randn(N, D_in)
x_torch = torch.randn(N, D_in, device=device, dtype=dtype)

In [3]:
x

array([[-0.16891271, -0.23268298, -0.74741032, ...,  0.35004517,
         1.59885609, -0.41753603],
       [-0.44573733, -0.39661629,  1.36451307, ..., -0.49599062,
         0.34722624,  0.43470394],
       [-2.17558904, -0.49547849, -0.14811798, ..., -0.51997495,
        -0.70286749,  1.52708251],
       ...,
       [ 0.8646832 ,  0.72373959, -0.67574586, ...,  0.91998562,
         0.6790937 ,  0.45249521],
       [-0.30208713, -0.18747416,  0.10906676, ...,  0.12930308,
        -0.54037368,  1.00312018],
       [ 0.4378578 ,  0.17303408, -0.09840088, ...,  1.13569445,
         0.78739593,  1.94283141]])

In [4]:
x_torch

tensor([[ 1.6007,  0.9988,  1.3070,  ..., -0.2944, -0.2131, -0.5551],
        [-1.1627,  0.6697, -1.2762,  ..., -0.3680, -0.9347,  0.7856],
        [-1.0388,  0.0380,  1.1237,  ..., -0.0821, -1.0454, -0.5834],
        ...,
        [-2.3841, -0.9967,  0.9198,  ...,  0.3005,  0.8582, -1.4633],
        [-0.7996,  0.1932, -0.5320,  ...,  1.8075,  0.4545, -0.2070],
        [-0.4868, -0.8683, -0.9240,  ...,  1.0124,  1.1584, -0.0568]])

In [5]:
x_torch = torch.Tensor(np.ones((N, D_in)))
x_torch

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [6]:
x_torch = torch.FloatTensor([1, 2, 3])
x_torch

tensor([1., 2., 3.])

In [7]:
x1 = torch.IntTensor([1, 2, 3])
x2 = torch.FloatTensor([3, 4, 5])

В PyTorch можно найти много операций, которые похожи на то, что есть в numpy :
```
- torch.add (np.add) -> сложение тензоров (поэлементное)
- torch.sub (np.subtract) -> вычитание (поэлементное)
- torch.mul (np.multiply) -> умнажение скаляров / матриц (поэлементное)
- torch.mm (np.matmul) -> перемножение матриц
- torch.ones (np.ones) -> создание тензора из единиц
```

In [8]:
# Давайте попробуем вышепересчисленные операции

In [9]:
x1 = torch.FloatTensor([[1, 2, 3], [4, 5, 6]])

In [10]:
x2 = torch.FloatTensor([[7, 8], [9, 1], [2, 3]])

In [11]:
out = torch.mm(x1, x2)
out

tensor([[31., 19.],
        [85., 55.]])

```
- torch.reshape (np.reshape) -> изменения порядка элементов в тензоре, не путать с транспонированием.
```

## Dynamic Computational Graph

После того, как были реализованы архитектура модели и весь процес обучения и валидация сети, при запуске кода в PyTorch происходят следующие этапы:

1. Строится вычислительный граф (направленный ациклический граф), где каждый узел -- это тензор, а ребро, ведущее к дргуому узлу, это выполнение операции над данным тензором, которое ведет к результату - другому тензору.

<img src="./images/Graph.png" alt="Drawing" style="width: 300px;"/>

Реализуем двухслойную сеть для задачи регрессии. И граф для такой архитектуры бдует выглядить следующим образом:

<img src="./images/RegGraph.png" alt="Drawing" />

In [53]:
batch_size = 64
input_size = 3
hidden_size = 2
output_size = 1

In [54]:
# Create random input and output data
x = torch.randn(batch_size, input_size, device=device, dtype=dtype)
y = torch.randn(batch_size, output_size, device=device, dtype=dtype)

# Randomly initialize weights AND BIAS
w1 = torch.randn(input_size, hidden_size, device=device, dtype=dtype)
b1 = torch.randn(1,hidden_size, device=device, dtype=dtype)
w2 = torch.randn(hidden_size, output_size, device=device, dtype=dtype)
b2 = torch.randn(1,output_size, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    h = torch.mm(x, w1) + b1 #матричное умножение x @ w1 + bias1
    h_relu = torch.clamp(h, min=0) 
    
    y_pred = torch.mm(h_relu,w2) + b2
    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()

## Autograd

2. Еще одно фундаментальное понятие и важный элемент при построении графа -- это __Autograd__ -- автоматическое дифференцирование.

Для того чтобы с помощью стохастического градиентного спуска обновить обучаемые параметры сети, нужно посчитать градиенты. И как известно, обновление весов, которые учавтсвуют в нескольких операциях, происходит по `правилу дифференцирования сложной функции` (цепное правило или __chain rule__).

<img src="./images/RegChainRule.png" alt="Drawing" />

То есть (1) вычислительный граф позволяет определить последовательность операций, а (2) автоматическое дифференцирование посчитать нужные градиенты.

Если бы `Autograd` не было, то тогда backprop надо было бы реализовывать самим, и как это бы выглядело?

Рассмотрим на примере, как посчиать градиенты для весов из входного слоя, где входной вектора `X` состоит из 3-х компонент. А входной слой вторую размерность имеет равной 2. 

После чего это идет в `ReLU`, но для простоты опустим на время ее, и посмотрим как дальше это идет по сети.

Ниже написано, как это все вычисляется и приводит нас к значению целевой функции для одного наблюдения

<img src="./images/1.png" alt="1" style="width: 600px;"/>

Тогда, чтобы посчитать градиент по первому элементу из обучаемой матрицы на первом слое, необходимо взять производоную у сложной функции. А этот как раз делается по `chain rule`: сначала берем у внешней, потом спускаемся на уровень ниже, и так пока не додйдем до то функции, после которой эта перменная уже нигде не участвует:

<img src="./images/2.png" alt="2" style="width: 400px;"/>

Перепишем это все в матричном виде, то есть сделаем аналог вида матрицы весов из первого слоя, но там уже будут её градиенты, котоыре будут нужны чтобы как раз обновить эти веса:

<img src="./images/3.jpg" alt="3" style="width: 600px;"/>

Как видно, здесь можно вектор X вынести, то есть разделить на две матрицы:

<img src="./images/4.jpg" alt="4" style="width: 500px;"/>

То есть уже видно, что будем траспонировать входной вектор(матрицу). Но надо понимать, что в реальности у нас не одно наблюдение в батче, а несколько, тогда запись немного изменит свой вид:

<img src="./images/5.jpg" alt="5" style="width: 500px;"/>

Теперь мы видим, как на самом деле вычисляется вот те самые частные производные для вектора X, то есть видно, как математически это можно записать, а именно:

<img src="./images/6.jpg" alt="6" style="width: 500px;"/>

<img src="./images/7.jpg" alt="7" style="width: 500px;"/>

Уже можно реализовать. Понятно, что транспонируется, что нет, и что на что умножается.

Но помним про ReLU. Для простоты опустили, но теперь её учесть будет легче. 

Так как после первого слоя идет ReLU, а значит, занулились те выходы первого слоя, которые были __меньше__ нуля. Получается, что во второй слой не все дошло, тогда нужно обнулить, что занулил ReLU. 

Что занулил ReLU, мы можем выяснить при `forward pass`, а где именно поставить нули, то надо уже смотреть относительно `backward propagation`, на том выходе, где последний раз участвовал выход после ReLU, то есть:

<img src="./images/8.jpg" alt="8" style="width: 600px;"/>

Теперь реализуем эти формулы на PyTorch:

In [55]:
for t in range(500):
    # Forward pass: compute predicted y
    h = torch.mm(x, w1) + b1 #матричное умножение x @ w1 + bias1
    h_relu = torch.clamp(h, min=0) 
    
    y_pred = torch.mm(h_relu,w2) + b2
    
    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)
    
    # loss.backward()
    #TODO
    s = 2 * (y_pred - y)
    grad_w2 = torch.mm(h_relu.T, s)
    grad_b2 = torch.mm(torch.ones(1,batch_size), s)

    # TODO
    grad_s = torch.mm(s, w2.T)
    grad_s[h_relu <= 0] = 0.
    grad_w1 = torch.mm(x.T, grad_s) 
    grad_b1 = torch.mm(torch.ones(1,batch_size), grad_s)


    #optimizer.step()
    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2
    b2 -= learning_rate * grad_b2
    b1 -= learning_rate * grad_b1


99 80.80897521972656
199 77.98828887939453
299 75.56156921386719
399 73.46370697021484
499 71.64057159423828


In [46]:
b2.shape

torch.Size([1, 1])

In [47]:
s.shape

torch.Size([64, 1])

In [44]:
grad_s.shape

torch.Size([64, 2])

Благодаря `Autograd` реализацию `chain rule` можно избежать, так как для более сложных нейронных сетей вручную такое реализовать сложно, при этом сделать это эффективным.

Для того чтобы PyTorch понял, за какими переменными надо "следить", то есть указать, что именно "эти" переменные являются обучаемыми, необходимо при создании тензора в качестве аттрибута указать __requires_grad=True__:

In [56]:
w1 = torch.randn(input_size, hidden_size, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(hidden_size, output_size, device=device, dtype=dtype, requires_grad=True)
b1 = torch.randn(1,hidden_size, device=device, dtype=dtype, requires_grad=True)
b2 = torch.randn(1,output_size, device=device, dtype=dtype, requires_grad=True)


In [57]:
learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    h = torch.mm(x, w1) + b1 #матричное умножение x @ w1 + bias1
    h_relu = torch.clamp(h, min=0) 
    
    y_pred = torch.mm(h_relu,w2) + b2

    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
    
    # Теперь подсчет градиентов для весов происходит при вызове backward
    loss.backward()
   
    # Обновляем значение весов, но укзаываем, чтобы PyTorch не считал эту операцию, 
    # которая бы учавствовала бы при подсчете градиентов в chain rule
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        b1 -= learning_rate * b1.grad
        b2 -= learning_rate * b2.grad
        
        # Теперь обнуляем значение градиентов, чтобы на следующем шаге 
        # они не учитывались при подсчете новых градиентов,
        # иначе произойдет суммирвоание старых и новых градиентов
        w1.grad.zero_()
        w2.grad.zero_()
        b1.grad.zero_()
        b2.grad.zero_()

99 79.49066925048828
199 76.53870391845703
299 73.97100830078125
399 71.7332992553711
499 69.77947998046875


Осталось еще не вручную обновлять веса, а использовать адаптивные методы градинетного спсука. Для этого нужно использовать модуль __optim__. А помимо оптимайзера, еще можно использовать готовые целевые функции из модлуя __nn__.

In [23]:
import torch.optim as optim

loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
optimizer = torch.optim.Adam([w1, w2, b1, b2], lr=learning_rate)

for t in range(500):
    optimizer.zero_grad()
    
    h = torch.mm(x, w1) + b1 #матричное умножение x @ w1 + bias1
    h_relu = torch.clamp(h, min=0) 
    
    y_pred = torch.mm(h_relu,w2) + b2
    
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
    
    loss.backward()
   
    optimizer.step()

99 83.53846740722656
199 83.53125
299 83.5240478515625
399 83.516845703125
499 83.50965881347656


После того, как мы сделали backward, в этот момент посчитались градиенты и граф уничтожился, то есть стёрлись все пути, которые связывали тензоры между собой. Это значит, что еще раз backward сделать не поулчится, будет ошибка. Но если вдруг нужно считать градиенты еще раз, то нужно при вызове backward задать `retain_graph=True`.

Еще важный аттрибут, который есть у Tensor -- это `grad_fn`. В этом аттрибуте указывается та функция, посредством которой был создан этот тензор. Так PyTorch понимает, как именно считать по нему градиент.

In [24]:
y_pred.grad_fn

Также можно контролировать, должны ли градиенты течь или нет.

In [25]:
x = torch.tensor([1], requires_grad=True)
with torch.no_grad():
    with torch.enable_grad():
        y = x * 2
y.requires_grad

RuntimeError: Only Tensors of floating point and complex dtype can require gradients

## Почему Backprop надо понимать

1. Backprop позволяет понимать, как те или иные операции, сложные конструкции в сети влияют на обнолвение весов.
Почему лучше сделать конкатенацию тензоров, а не поэлементное сложение. Для этого нужно посмотреть на backprop, как будут обновляться веса.

2. Даже на таком маленьком пример двуслойной MLP можно уже увидеть, когда `ReLU`, как функция активация, не очень хорошо применять. Если разреженные данные, то получить на выходе много нулей вероятнее, чем при использовании `LeakyReLU`, то есть градиенты будут нулевыми и веса никак не будут обновляться => сеть не обучается!

3. В архитектуре могут встречаться недифференцируемые операции, и первое - это нужно понять, потому что при обучении сети это может быть не сразу заметно, просто качество модели будет плохое, и точность хорошую не поулчится достичь.

Например, в одной из статей было предложено в качестве механизма внимания применить распределение Бернулли, которое умножается на выход промежуточного слоя сети. И эта опреация недифференцируема, нужно реализовывать backprop самим, тем самым обеспечить корректное протекание градиентов.


<img src="./images/Bernoulli.png" alt="8" style="width: 600px;"/>

Так же любая статья, которая предлагет новую целевую функцию для той или иной задачи, там всегда будут представлены градиенты, чтобы было понимание, как это влияет на обновление весов. Не просто так !

<img src="./images/BernoulliBackProp.png" alt="8" style="width: 600px;"/>

## nn.Module

В предыдущем примере архитектуру сети создавали используя последовательной способ объявления слоев сети -- `nn.Sequential`.

Но еще можно это сделать более гибким подходом:

In [27]:
import torch.nn as nn
class TwoLayerNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        """
        TwoLayerNet наследуется от nn.Module и тем самым полчаем возможность
        переопределять методы класса.
        В конструктуре создаем слои (обучаемые веса) и другие нужные перменные/функции,
        которые нужны для модели
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(input_size, hidden_size, bias = True)
        self.linear2 = torch.nn.Linear(hidden_size, output_size, bias = True)

    def forward(self, x):
        """
        Метод forward отвечает за прямое распростронение модели, 
        поэтому данный метод нужно переопределять обязательно, 
        чтобы задать логику прямого распростронения. 
        Именно в этот момент начинает строится динамический граф
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        
        return y_pred

In [29]:
batch_size = 64
input_size = 1000
hidden_size = 100
output_size = 10

x = torch.randn(batch_size, input_size, device=device, dtype=dtype)
y = torch.randn(batch_size, output_size, device=device, dtype=dtype)

model = TwoLayerNet(input_size, hidden_size, output_size)

loss_fn = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

for t in range(500):
    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 2.820061683654785
199 0.06132138520479202
299 0.0024999494198709726
399 0.00013461268099490553
499 8.467967745673377e-06
